# The goal of this project is to extact data from an API and store it in AWS RDS

# Import libraries

In [2]:
pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 2.2 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd
import pymysql
import datetime

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Extarct all data from API in json format

In [4]:
# Documentation: https://coinmarketcap.com/api/documentation/v1/#operation/getV1CryptocurrencyMap

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
api_key = '83f659b5-cb23-45e9-87f2-733a317f404d'

parameters = {
  'start':'1',
  'limit':'5000',
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': api_key,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Show data

In [ ]:
data['data'][0:2]

[{'id': 1,
  'name': 'Bitcoin',
  'symbol': 'BTC',
  'slug': 'bitcoin',
  'num_market_pairs': 9976,
  'date_added': '2013-04-28T00:00:00.000Z',
  'tags': ['mineable',
   'pow',
   'sha-256',
   'store-of-value',
   'state-channel',
   'coinbase-ventures-portfolio',
   'three-arrows-capital-portfolio',
   'polychain-capital-portfolio',
   'binance-labs-portfolio',
   'blockchain-capital-portfolio',
   'boostvc-portfolio',
   'cms-holdings-portfolio',
   'dcg-portfolio',
   'dragonfly-capital-portfolio',
   'electric-capital-portfolio',
   'fabric-ventures-portfolio',
   'framework-ventures-portfolio',
   'galaxy-digital-portfolio',
   'huobi-capital-portfolio',
   'alameda-research-portfolio',
   'a16z-portfolio',
   '1confirmation-portfolio',
   'winklevoss-capital-portfolio',
   'usv-portfolio',
   'placeholder-ventures-portfolio',
   'pantera-capital-portfolio',
   'multicoin-capital-portfolio',
   'paradigm-portfolio'],
  'max_supply': 21000000,
  'circulating_supply': 19289625,
  '

# Create dataframe to store all data

In [5]:
# Create dataframe to store all data

columns = ['id', 'name', 'symbol', 'last_updated', 'circulating_supply',
          'total_supply', 'max_supply', 'price', 'volume_24h', 'percent_change_1h', 'percent_change_7d']

df = pd.DataFrame(columns = columns)

# Fill in df

for coin in data['data']:
    id = coin['id']
    name = coin['name']
    symbol = coin['symbol']
    last_updated = coin['last_updated']
    circulating_supply = coin['circulating_supply']
    total_supply = coin['total_supply']
    max_supply = coin['max_supply']
    price = coin['quote']['USD']['price']
    volume_24h = coin['quote']['USD']['volume_24h']
    percent_change_1h = coin['quote']['USD']['percent_change_1h']
    percent_change_24h = coin['quote']['USD']['percent_change_24h']
    percent_change_7d = coin['quote']['USD']['percent_change_7d']

    data_dict = {'id': id, 
                'name': name, 
                'symbol': symbol,
                'last_updated': last_updated,
                'circulating_supply': circulating_supply,
                'total_supply': total_supply,
                'max_supply': max_supply,
                'price': price,
                'volume_24h': volume_24h,
                'percent_change_1h': percent_change_1h,
                'percent_change_7d': percent_change_7d}

    df = df.append(data_dict, ignore_index = True)
  


# Show df

In [6]:
df.head()

,id,name,symbol,last_updated,circulating_supply,total_supply,max_supply,price,volume_24h,percent_change_1h,percent_change_7d
0,1,Bitcoin,BTC,2023-02-14T11:38:00.000Z,19290518,19290518,21000000,21824.359825,2.086635e+10,-0.112827,-5.051905
1,1027,Ethereum,ETH,2023-02-14T11:38:00.000Z,122373866.2178,122373866.2178,None,1509.020064,6.916400e+09,-0.326357,-8.176212
2,825,Tether,USDT,2023-02-14T11:38:00.000Z,68411642034.939713,73141766321.234283,None,1.000931,3.216217e+10,-0.015542,0.079888
3,1839,BNB,BNB,2023-02-14T11:38:00.000Z,157898631.62932,159979963.590429,200000000,292.698540,7.536730e+08,-0.320278,-11.410475
4,3408,USD Coin,USDC,2023-02-14T11:38:00.000Z,41043894229.094063,41043894229.094063,None,0.999906,3.801351e+09,-0.016406,-0.011452


In [ ]:
df.columns

Index(['id', 'name', 'symbol', 'last_updated', 'circulating_supply',
       'total_supply', 'max_supply', 'price', 'volume_24h',
       'percent_change_1h', 'percent_change_7d'],
      dtype='object')

# Check data quality of df

In [7]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
  '''
    1) Definition: Function used to check data quality of df
    
    2) Input:
            df: dataframe whose columns we'll check (type = pd.Dataframe)

    3) Return: 
            True = Data Quality approved
            False = Data Quality reproved
    '''
    
  # Check if dataframe is empty

  if df.empty:
      print("\nDataframe empty. Finishing execution")
      return False 

  # Check for nulls in symbol column

  if df.symbol.empty:
      raise Exception("\nSymbol is Null or the value is empty")

    # Check for nulls in price column

  if df.price.empty:
      raise Exception("\nPrice is Null or the value is empty")

  # Check for nulls in data_added column

  if df.data_added.empty:
      raise Exception("\nData is Null or the value is empty")

  return True

In [ ]:
check_if_valid_data(df)

True

# Load data to AWS RDS

In [8]:
# Create connection to AWS database

host = 'crypto.csyqer7pwgvm.us-east-1.rds.amazonaws.com'
port = 3306
user = 'admin'
password = 'stackdataengineer'

connection = pymysql.connect(host = host,
                             port = port,
                             user = user,
                             passwd = password)
 
cursor = connection.cursor()

In [ ]:
# Create data base
# On AWS, we create an AWS RDS data base instance

query = '''create database crypto_data_base'''
cursor.execute(query)
cursor.connection.commit()

In [ ]:
# Connect to data base

query = '''use crypto_data_base'''
cursor.execute(query)

# Create table

query = '''create table crypto_table (
              id int, 
              name varchar(50), 
              symbol varchar(50), 
              last_updated date,
              circulating_supply float, 
              total_supply float, 
              max_supply float, 
              price float,
              volume_24h float, 
              percent_change_1h float, 
              percent_change_7d float 
              )
              '''

cursor.execute(query)

0

In [10]:
# Connect to data base

query = '''use crypto_data_base'''
cursor.execute(query)

# Show tables

query = '''show tables'''
cursor.execute(query)
cursor.fetchall()

(('crypto_table',),)

In [11]:
# Check if we have data

query = '''select * from crypto_table'''
cursor.execute(query)
cursor.fetchall()

((1,
  'Bitcoin',
  'BTC',
  datetime.date(2023, 2, 13),
  19289800.0,
  19289800.0,
  21000000.0,
  21671.9,
  24115500000.0,
  0.887486,
  -5.78859),
 (1027,
  'Ethereum',
  'ETH',
  datetime.date(2023, 2, 13),
  122374000.0,
  122374000.0,
  None,
  1491.53,
  9678930000.0,
  1.32602,
  -9.31385),
 (825,
  'Tether',
  'USDT',
  datetime.date(2023, 2, 13),
  68411600000.0,
  73141800000.0,
  None,
  1.00103,
  41183200000.0,
  -0.00742384,
  0.0909754),
 (1839,
  'BNB',
  'BNB',
  datetime.date(2023, 2, 13),
  157899000.0,
  159980000.0,
  200000000.0,
  288.951,
  1328620000.0,
  1.56833,
  -12.04),
 (3408,
  'USD Coin',
  'USDC',
  datetime.date(2023, 2, 13),
  40873500000.0,
  40873500000.0,
  None,
  1.00011,
  4815200000.0,
  0.018931,
  0.0059342),
 (52,
  'XRP',
  'XRP',
  datetime.date(2023, 2, 13),
  50799100000.0,
  99989100000.0,
  100000000000.0,
  0.370072,
  1124910000.0,
  1.42508,
  -8.09182),
 (4687,
  'Binance USD',
  'BUSD',
  datetime.date(2023, 2, 13),
  15873300

In [ ]:
# check data types from crypto_table

query = query = '''SHOW FIELDS FROM crypto_table FROM crypto_data_base'''

cursor.execute(query)
cursor.fetchall()

(('id', 'int', 'NO', 'PRI', None, ''),
 ('name', 'varchar(50)', 'YES', '', None, ''),
 ('symbol', 'varchar(50)', 'YES', '', None, ''),
 ('last_updated', 'date', 'YES', '', None, ''),
 ('circulating_supply', 'float', 'YES', '', None, ''),
 ('total_supply', 'float', 'YES', '', None, ''),
 ('max_supply', 'float', 'YES', '', None, ''),
 ('price', 'float', 'YES', '', None, ''),
 ('volume_24h', 'float', 'YES', '', None, ''),
 ('percent_change_1h', 'float', 'YES', '', None, ''),
 ('percent_change_7d', 'float', 'YES', '', None, ''))

In [ ]:
# Insert data from df

for index, row in df.iterrows():
    print(f'\nindex = {index} / {len(df)}')

    query = '''INSERT INTO crypto_table

               (id, name, symbol, last_updated, circulating_supply, total_supply, 
                max_supply, price, volume_24h, percent_change_1h, percent_change_7d) 

               values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

    id = int(row['id']) if row['id'] is not None else None
    name = str(row['name']) if row['name'] is not None else None 
    symbol = str(row['symbol']) if row['symbol'] is not None else None
    last_updated = row['last_updated'][0:10]
    last_updated = datetime.datetime.strptime(last_updated, '%Y-%m-%d') if row['last_updated'] is not None else None
    circulating_supply = float(row['circulating_supply']) if row['circulating_supply'] is not None else None 
    total_supply = float(row['total_supply']) if row['total_supply'] is not None else None
    max_supply = float(row['max_supply']) if row['max_supply'] is not None else None 
    price = float(row['price']) if row['price'] is not None else None 
    volume_24h = float(row['volume_24h']) if row['volume_24h'] is not None else None
    percent_change_1h = float(row['percent_change_1h']) if row['percent_change_1h'] is not None else None
    percent_change_7d = float(row['percent_change_7d']) if row['percent_change_7d'] is not None else None
  
    values = id, name, symbol, last_updated, circulating_supply, total_supply, \
             max_supply, price, volume_24h, percent_change_1h, percent_change_7d 

    cursor.execute(query, values)

connection.commit()

A saída de streaming foi truncada nas últimas 5000 linhas.

index = 2500 / 5000

index = 2501 / 5000

index = 2502 / 5000

index = 2503 / 5000

index = 2504 / 5000

index = 2505 / 5000

index = 2506 / 5000

index = 2507 / 5000

index = 2508 / 5000

index = 2509 / 5000

index = 2510 / 5000

index = 2511 / 5000

index = 2512 / 5000

index = 2513 / 5000

index = 2514 / 5000

index = 2515 / 5000

index = 2516 / 5000

index = 2517 / 5000

index = 2518 / 5000

index = 2519 / 5000

index = 2520 / 5000

index = 2521 / 5000

index = 2522 / 5000

index = 2523 / 5000

index = 2524 / 5000

index = 2525 / 5000

index = 2526 / 5000

index = 2527 / 5000

index = 2528 / 5000

index = 2529 / 5000

index = 2530 / 5000

index = 2531 / 5000

index = 2532 / 5000

index = 2533 / 5000

index = 2534 / 5000

index = 2535 / 5000

index = 2536 / 5000

index = 2537 / 5000

index = 2538 / 5000

index = 2539 / 5000

index = 2540 / 5000

index = 2541 / 5000

index = 2542 / 5000

index = 2543 / 5000

index = 2544 / 5

In [ ]:
# Select all data from AWS RDS

query = '''select * from crypto_table'''
cursor.execute(query)
cursor.fetchall()


((1,
  'Bitcoin',
  'BTC',
  datetime.date(2023, 2, 13),
  19289800.0,
  19289800.0,
  21000000.0,
  21671.9,
  24115500000.0,
  0.887486,
  -5.78859),
 (1027,
  'Ethereum',
  'ETH',
  datetime.date(2023, 2, 13),
  122374000.0,
  122374000.0,
  None,
  1491.53,
  9678930000.0,
  1.32602,
  -9.31385),
 (825,
  'Tether',
  'USDT',
  datetime.date(2023, 2, 13),
  68411600000.0,
  73141800000.0,
  None,
  1.00103,
  41183200000.0,
  -0.00742384,
  0.0909754),
 (1839,
  'BNB',
  'BNB',
  datetime.date(2023, 2, 13),
  157899000.0,
  159980000.0,
  200000000.0,
  288.951,
  1328620000.0,
  1.56833,
  -12.04),
 (3408,
  'USD Coin',
  'USDC',
  datetime.date(2023, 2, 13),
  40873500000.0,
  40873500000.0,
  None,
  1.00011,
  4815200000.0,
  0.018931,
  0.0059342),
 (52,
  'XRP',
  'XRP',
  datetime.date(2023, 2, 13),
  50799100000.0,
  99989100000.0,
  100000000000.0,
  0.370072,
  1124910000.0,
  1.42508,
  -8.09182),
 (4687,
  'Binance USD',
  'BUSD',
  datetime.date(2023, 2, 13),
  15873300